<a href="https://colab.research.google.com/github/Rabiahdlearner/SQL/blob/main/CFG_SQL_Pizza_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Some Pizza Sales Analytics with MySQL





In [14]:
#@title Setup { display-mode: "form" }
#@markdown Run this cell to prepare MySQL. Rerun the cell whenever you start a new session, or if you think your database might have shut down. **Do not edit its contents.**
#
#@markdown *If any errors occur upon running this cell, please rerun the cell to see if that fixes it. If not, ask for help. Remember to attach a screenshot.*
!printf "Installing MySQL server... "; apt-get install mysql-server &>/dev/null 2>/dev/null && echo -e "success." || echo "failure!"
!printf "Starting MySQL server... "; service mysql start &>/dev/null 2>/dev/null && echo "success." || echo "failure!"
!printf "Installing MySQL client... "; pip install mysqlclient &>/dev/null 2>/dev/null && echo "success." || echo "failure!"
!printf "Creating database connection for root user... "
import IPython, sqlalchemy, time; sqlalchemy.create_engine("mysql://root@localhost/")
%reload_ext sql
%sql mysql://root@localhost/ ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY ''; FLUSH PRIVILEGES; SET sql_mode = (SELECT REPLACE(@@sql_mode, 'ONLY_FULL_GROUP_BY', ''));
!printf "Setup attempt finished. Please retry or ask for help if there were any errors."

Installing MySQL server... success.
Starting MySQL server... success.
Installing MySQL client... success.
Creating database connection for root user... 0 rows affected.
0 rows affected.
0 rows affected.
Setup attempt finished. Please retry or ask for help if there were any errors.

This project makes use of Pizza sales of dataset of an unstated location in the year 2015, which is available
to the public found on Kaggle website  it consists of 48620 rows and 12 columns
and is divided into five tables namely:
1. features
2. order
3. Price
4. ingredients
5. November Sales
 to fulfill this requirement of this project as well as provide clarity for potential users of the dataset. Hence the name of the Database for this table will be call "Pizza".

In [15]:
!pip install mysql-connector-python


Connect Mysql to Google collaboratory

In [16]:
import mysql.connector

Initiate Server connection with local host

In [17]:
pizza = mysql.connector.connect(host="localhost", user="root")#, password="")

Create Pizza Database where all tables and others will be stored

In [18]:
%%sql
CREATE DATABASE pizza;

 * mysql://root@localhost/
(MySQLdb.ProgrammingError) (1007, "Can't create database 'pizza'; database exists")
[SQL: CREATE DATABASE pizza;]
(Background on this error at: https://sqlalche.me/e/20/f405)


Use the database created. this step is important so that items will nt get stored in another database if you had one initialized before

In [19]:
%%sql
USE pizza;

 * mysql://root@localhost/
0 rows affected.


[]

In [9]:
%%sql
CREATE TABLE pizza_sales ( pizza_id INT, order_id INT, pizza_name_id VARCHAR(15),
            quantity INT, order_date VARCHAR(12), order_time VARCHAR(12),
            unit_price DECIMAL(8, 2), total_price DECIMAL(8, 2), pizza_size VARCHAR(5),
            pizza_category VARCHAR(10), pizza_ingredients VARCHAR(255),
            pizza_name VARCHAR(50)
             );


 * mysql://root@localhost/
0 rows affected.


[]

The dataset  brought into google collab is now being stored in server

In [10]:
import csv

with open ("pizzaSales.csv") as file:
  rows = csv.DictReader(file)
  cursor = pizza.cursor()
  for row in rows:
    cursor.execute("""INSERT INTO pizza.pizza_sales( pizza_id, order_id, pizza_name_id,
            quantity, order_date, order_time,
            unit_price, total_price, pizza_size,
            pizza_category, pizza_ingredients,
            pizza_name
             ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""",
                   [int(row["pizza_id"]),
                    int(row["order_id"]),
                    row["pizza_name_id"],
                    int(row["quantity"]),
                    row["order_date"],
                    row["order_time"],
                    float(row["unit_price"]),
                    float(row["total_price"]),
                    row["pizza_size"],
                    row["pizza_category"],
                    row["pizza_ingredients"],
                    row["pizza_name"]
                    ])
  pizza.commit()

Verifying appropriate loading of the dataset into the database

In [20]:
%%sql
SELECT * FROM pizza_sales
LIMIT 5

 * mysql://root@localhost/
5 rows affected.


pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
1,1,hawaiian_m,1,1/1/2015,11:38:36,13.25,13.25,M,Classic,Sliced_Ham__Pineapple__Mozzarella_Cheese,The_Hawaiian_Pizza
2,2,classic_dlx_m,1,1/1/2015,11:57:40,16.00,16.00,M,Classic,Pepperoni__Mushrooms__Red_Onions__Red_Peppers__Bacon,The_Classic_Deluxe_Pizza
3,2,five_cheese_l,1,1/1/2015,11:57:40,18.50,18.50,L,Veggie,Mozzarella_Cheese__Provolone_Cheese__Smoked_Gouda_Cheese__Romano_Cheese__Blue_Cheese__Garlic,The_Five_Cheese_Pizza
4,2,ital_supr_l,1,1/1/2015,11:57:40,20.75,20.75,L,Supreme,Calabrese_Salami__Capocollo__Tomatoes__Red_Onions__Green_Olives__Garlic,The_Italian_Supreme_Pizza
5,2,mexicana_m,1,1/1/2015,11:57:40,16.00,16.00,M,Veggie,Tomatoes__Red_Peppers__Jalapeno_Peppers__Red_Onions__Cilantro__Corn__Chipotle_Sauce__Garlic,The_Mexicana_Pizza


W

1. Pizza feature table

In [12]:
%%sql
CREATE TABLE features
SELECT pizza_id, pizza_name_id, pizza_size, pizza_category
FROM pizza_sales;

 * mysql://root@localhost/
48620 rows affected.


[]

Add key constraint to features

In [21]:
%%sql
ALTER TABLE features
	ADD PRIMARY KEY (pizza_id);



 * mysql://root@localhost/
(MySQLdb.OperationalError) (1068, 'Multiple primary key defined')
[SQL: ALTER TABLE features
	ADD PRIMARY KEY (pizza_id);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


2. order details table

In [22]:
%%sql

CREATE TABLE orders
	SELECT pizza_id, pizza_name_id, pizza_category, quantity, STR_TO_DATE(order_date, '%d/%m/%Y') AS order_date,
		   order_time, total_price
	  FROM pizza_sales;



 * mysql://root@localhost/
48620 rows affected.


[]

Add KEY constraint to orders table

In [23]:
%%sql
/*throwing error when adding foreign so
i want to try it after creating the table*/
ALTER TABLE orders
	ADD FOREIGN KEY features (pizza_id)
		REFERENCES features (pizza_id);


 * mysql://root@localhost/
48620 rows affected.


[]

3. Create table price

In [24]:
%%sql
CREATE TABLE price
	  SELECT pizza_id, pizza_category, quantity, unit_price, total_price
        FROM pizza_sales;


 * mysql://root@localhost/
48620 rows affected.


[]

ADD key constraint to price table

In [25]:
%%sql
ALTER TABLE price
    ADD FOREIGN KEY ( pizza_id) REFERENCES features (pizza_id);


 * mysql://root@localhost/
48620 rows affected.


[]

4. Create table ingredients

In [27]:
%%sql
-- 4. Ingredients
CREATE TABLE ingredients
	SELECT pizza_id, pizza_category, pizza_ingredients AS pizza_ing
    FROM pizza_sales;




 * mysql://root@localhost/
(MySQLdb.OperationalError) (1050, "Table 'ingredients' already exists")
[SQL: -- 4. Ingredients
CREATE TABLE ingredients
	SELECT pizza_id, pizza_category, pizza_ingredients AS pizza_ing
    FROM pizza_sales;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Add KEY constraints to ingredients table

In [28]:
%%sql
ALTER TABLE ingredients
    ADD FOREIGN KEY features (pizza_id) REFERENCES features (pizza_id);

 * mysql://root@localhost/
48620 rows affected.


[]

5. Create table November sales

In [29]:
%%sql
CREATE TABLE nov_sales
SELECT  STR_TO_DATE(order_date, '%d/%m/%Y') AS order_date, pizza_id, pizza_category,
        CAST(order_time AS TIME), quantity, total_price
        FROM pizza_sales;
		-- WHERE order_date LIKE '%/11/2015%';
 -- SELECT  *FROM nov_sales; -- DATE -- (order_date)='29/11/2015'
 -- drop table nov_sales;




 * mysql://root@localhost/
48620 rows affected.
0 rows affected.


[]

Add key constraint to november sales table

In [30]:
%%sql
ALTER TABLE nov_sales
	 ADD FOREIGN KEY (pizza_id) REFERENCES  features (pizza_id);

 * mysql://root@localhost/
48620 rows affected.


[]

the subsequent  code chunk will look into total number of pizza type sold on black friday and then we will comapre the quantity a day before and after blackfiday respectively

In [33]:
%%sql
CREATE VIEW blkfri AS
SELECT pizza_category, COUNT(quantity) AS blackfriday_qty
	    FROM nov_sales
	   WHERE DATE(order_date) = '2015-11-29'
       GROUP BY pizza_category;

 * mysql://root@localhost/
0 rows affected.


[]

In [31]:
  %%sql
CREATE VIEW b4_blkfri AS
  SELECT pizza_category, COUNT(quantity) AS dayb4_qty
    FROM nov_sales
   WHERE DATE(order_date) = '2015-11-28'
   GROUP BY pizza_category;

 * mysql://root@localhost/
0 rows affected.


[]

In [32]:
%%sql
CREATE VIEW aft_blkfri AS
SELECT pizza_category, COUNT(quantity) AS dayaft_qty
  FROM nov_sales
  WHERE DATE(order_date) = '2015-11-30'
  GROUP BY pizza_category;

 * mysql://root@localhost/
0 rows affected.


[]

This did not work initially google collab but when the sql-mode is set to ''
such that we have

```
SET sql_mode = (SELECT REPLACE(@@sql_mode, 'ONLY_FULL_GROUP_BY', ''));
```
this work for any group by


In [34]:
%%sql
SELECT * FROM b4_blkfri
  INNER JOIN blkfri USING (pizza_category)
  INNER JOIN  aft_blkfri USING (pizza_category);


 * mysql://root@localhost/
4 rows affected.


pizza_category,dayb4_qty,blackfriday_qty,dayaft_qty
Classic,37,32,38
Chicken,29,17,43
Veggie,20,31,25
Supreme,30,31,26


From the above we can that black friday is not necessarily the highest pizza sales day, lets see which pizza sells best

In [26]:
%%sql
SELECT pizza_category, MAX(quantity) AS quantity, MONTH(order_date) AS order_month
	FROM orders
    GROUP BY pizza_category, MONTH(order_date) = order_month;


 * mysql://root@localhost/
4 rows affected.


pizza_category,quantity,order_month
Classic,4,1
Veggie,3,1
Supreme,3,1
Chicken,4,1


Lets now see the average cost of each pizza category sold per month


In [32]:
%%sql

   SELECT pizza_category, AVG(total_price) AS avg_price, MONTH(order_date) AS order_month
    FROM  orders
GROUP BY  pizza_category, MONTH(order_date);

 * mysql://root@localhost/
48 rows affected.


pizza_category,avg_price,order_month
Classic,15.039903,1
Veggie,17.089579,1
Supreme,17.475390,1
Chicken,18.108221,1
Classic,14.970725,2
Veggie,16.867745,2
Supreme,17.635918,2
Chicken,18.007276,2
Classic,14.960033,3
Chicken,17.966871,3


Comparing the price of chicken pizza across months

In [35]:
%%sql
   SELECT pizza_category, total_price, MONTH(order_date) AS month
	   FROM orders
    WHERE pizza_category LIKE "%Chicken"
 GROUP BY MONTH(order_date);

 * mysql://root@localhost/
12 rows affected.


pizza_category,total_price,month
Chicken,20.75,1
Chicken,20.75,2
Chicken,12.75,3
Chicken,20.75,4
Chicken,12.75,5
Chicken,16.75,6
Chicken,20.75,7
Chicken,16.75,8
Chicken,20.75,9
Chicken,12.75,10


We create function to calculate the proportion of pizza sold

In [42]:
%%sql

DELIMITER //

CREATE FUNCTION perc(quantity DECIMAL(5, 3))
RETURNS DECIMAL(5, 3)
NO SQL
BEGIN
  RETURN 100 * quantity / 48620;
END//

 DELIMITER ;

 * mysql://root@localhost/
(MySQLdb.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'DELIMITER//\n\nCREATE FUNCTION perc(quantity DECIMAL(5, 3))\nRETURNS DECIMAL(5, 3)\n' at line 1")
[SQL: DELIMITER//

CREATE FUNCTION perc(quantity DECIMAL(5, 3))
RETURNS DECIMAL(5, 3)
NO SQL
BEGIN
RETURN 100 * quantity / 48620;
END//

 DELIMITER ;]
(Background on this error at: https://sqlalche.me/e/20/f405)


The function as not worked here bur will continue

In [43]:
%%sql
SELECT  o.pizza_category, proportion(o.quantity) AS pizza_prop,
		MONTH(o.order_date)
        FROM orders AS o
        GROUP BY o.pizza_category, MONTH(order_date);



 * mysql://root@localhost/
(MySQLdb.OperationalError) (1305, 'FUNCTION pizza.proportion does not exist')
[SQL: SELECT  o.pizza_category, proportion(o.quantity) AS pizza_prop,
		MONTH(o.order_date)
        FROM orders AS o
        GROUP BY o.pizza_category, MONTH(order_date);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Lets check out the peak order time for for pizza per month. I want to use subquery here

In [36]:
%%sql

  SELECT o.pizza_name_id, o.pizza_category, o.order_time, MONTH(o.order_date) AS peak_month, DAY(o.order_date) AS peak_day
    FROM orders AS o
   WHERE o.quantity = (SELECT MAX(`or`.quantity) FROM orders AS `or`)
GROUP BY MONTH(order_date);

 * mysql://root@localhost/
3 rows affected.


pizza_name_id,pizza_category,order_time,peak_month,peak_day
big_meat_s,Classic,13:24:35,3,30
big_meat_s,Classic,12:42:46,5,19
cali_ckn_l,Chicken,14:16:35,9,19


What if i dont want to use a subquery

In [ ]:
%%sql
SELECT pizza_name_id, pizza_category, MAX(quantity),
       order_time AS peak_time,
       MONTH(order_date) AS peak_month, DAY(order_date) AS peak_day
	FROM orders
    GROUP BY  MONTH(order_date);

 * mysql://root@localhost/
12 rows affected.


pizza_name_id,pizza_category,MAX(quantity),peak_time,peak_month,peak_day
hawaiian_m,Classic,3,11:38:36,1,1
big_meat_s,Classic,3,11:32:07,2,1
pepperoni_m,Classic,4,11:34:00,3,1
sicilian_m,Supreme,2,11:17:06,4,1
classic_dlx_m,Classic,4,11:31:03,5,1
green_garden_m,Veggie,3,11:21:45,6,1
bbq_ckn_l,Chicken,3,11:16:34,7,1
bbq_ckn_m,Chicken,3,11:36:22,8,1
ital_cpcllo_l,Classic,4,11:18:24,9,1
pep_msh_pep_m,Classic,3,11:40:34,10,1


In conclusion, I am able to discern that chosing the query to use entail absolute care because our output sometimes might not represent our need, conversely it can pave way to interesting discovery as seen above.

However, as cliche as this may sound, it is always and forever be true that accuracy comes with practice.

# Thank You